<a href="https://colab.research.google.com/github/andyradz/aplios.solutions/blob/master/introduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AR-DiplomaWorkspace

#### MOUNT GOOGLE DRIVE VOLUME

In [3]:
## MOUNT GOOGLE DRIVE VOLUME

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### LIST GOOGLE DRIVE VOLUME

In [4]:
## LIST GOOGLE DRIVE VOLUME
!ls -laFtA --full-time /content/gdrive 

### CREATE REQUIRED DATA FOLDERS

!mkdir input
!mkdir output

total 16
dr-x------  2 root root 4096 2022-11-24 16:10:12.000000000 +0000 .file-revisions-by-id/
drwx------ 22 root root 4096 2022-11-24 16:10:12.000000000 +0000 MyDrive/
dr-x------  2 root root 4096 2022-11-24 16:10:12.000000000 +0000 .shortcut-targets-by-id/
drwx------  5 root root 4096 2022-11-24 16:10:12.000000000 +0000 .Trash-0/


In [5]:
### DOWNLOAD AND INSTALL JAVA VIRTUAL MACHINE(JVM)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
### DOWNLOAD APACHE SPARK AND HADOOP ENVIRONMENT
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz > /dev/null

### UNZIP SPARK ENVIRONMENT
!tar xf spark-3.3.1-bin-hadoop3.tgz > /dev/null

--2022-11-24 16:10:30--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M   214MB/s    in 1.3s    

2022-11-24 16:10:31 (214 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [7]:
### SET SYSTEM VARIABLES FOR PYSPARK

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.1-bin-hadoop3'
os.environ['PYSPARK_SUBMIT_ARGS'] = '---jars /content/spark-3.3.1-bin-hadoop3/jars/spark-xml_2.12-0.11.0.jar pyspark-shell'

In [8]:
### INSTALL PYSPARK ENVIRONMENT
!pip install -q pyspark > /dev/null

### INSTALL LIBRARY FIND PYSPARK
!pip install -q findspark > /dev/null

### INITIAL PYSPARK ENVIRONMENT
import findspark
findspark.init()

### CHECK PYSPARK LOCATION
findspark.find()


'/content/spark-3.3.1-bin-hadoop3'

In [9]:
!sudo apt install --assume-yes p7zip-full

!type(7z)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/bin/bash: -c: line 0: syntax error near unexpected token `7z'
/bin/bash: -c: line 0: `type(7z)'


In [10]:
### LIST PYTHON INSTALLED PACKAGES

!pip3 list -v


Package                       Version                Location                               Installer
----------------------------- ---------------------- -------------------------------------- ---------
absl-py                       1.3.0                  /usr/local/lib/python3.7/dist-packages pip
aeppl                         0.0.33                 /usr/local/lib/python3.7/dist-packages pip
aesara                        2.7.9                  /usr/local/lib/python3.7/dist-packages pip
aiohttp                       3.8.3                  /usr/local/lib/python3.7/dist-packages pip
aiosignal                     1.3.1                  /usr/local/lib/python3.7/dist-packages pip
alabaster                     0.7.12                 /usr/local/lib/python3.7/dist-packages pip
albumentations                1.2.1                  /usr/local/lib/python3.7/dist-packages pip
altair                        4.2.0                  /usr/local/lib/python3.7/dist-packages pip
appdirs                     

# Download stackexchange data 

In [11]:
#!FILE=/content/input/books.xml
#!if ![-f "$FILE"]; then
#!    echo Start downloading file $FILE
#!    wget https://github.com/databricks/spark-xml/raw/master/src/test/resources/books.xml  -P /content/input/ > /dev/null
#!fi


!wget https://archive.org/download/stackexchange/mathematica.stackexchange.com.7z -P /content/input/ > /dev/null

!7za e -y /content/input/mathematica.stackexchange.com.7z -o/content/input/ > /dev/null

--2022-11-24 16:11:22--  https://archive.org/download/stackexchange/mathematica.stackexchange.com.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800107.us.archive.org/27/items/stackexchange/mathematica.stackexchange.com.7z [following]
--2022-11-24 16:11:22--  https://ia800107.us.archive.org/27/items/stackexchange/mathematica.stackexchange.com.7z
Resolving ia800107.us.archive.org (ia800107.us.archive.org)... 207.241.232.17
Connecting to ia800107.us.archive.org (ia800107.us.archive.org)|207.241.232.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265738334 (253M) [application/x-7z-compressed]
Saving to: ‘/content/input/mathematica.stackexchange.com.7z’

mathematica.stackex 100%[===================>] 253.43M  4.29MB/s    in 32s     

2022-11-24 16:11:55 (7.96 MB/s) - ‘/content/input/mathematica.stackexchange.com

#### LOAD AND SAVE POSTS.XML DATASET TO PARQUET FORMAT

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, BooleanType, DateType 

spark = SparkSession.builder.master("local[4]").appName('SparkByExamples.com').config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.6.0').getOrCreate()


schema = StructType([ \
    StructField("_AnswerCount", LongType(), True), \
    StructField("_Body", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_LastActivityDate", StringType(), True), \
    StructField("_OwnerUserId", LongType(), True), \
    StructField("_ParentId", LongType(), True), \
    StructField("_PostTypeId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Tags", StringType(), True), \
    StructField("_Title", StringType(), True), \
    StructField("_ViewCount", LongType(), True) 
]) 

postsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posts') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Posts.xml')     

postsDF = postsDF \
                .withColumnRenamed("_AnswerCount", "AnswerCount") \
                .withColumnRenamed("_Body", "Body") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_LastActivityDate", "LastActivityDate") \
                .withColumnRenamed("_OwnerUserId", "OwnerUserId") \
                .withColumnRenamed("_ParentId", "ParentId") \
                .withColumnRenamed("_PostTypeId", "PostTypeId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Tags", "Tags") \
                .withColumnRenamed("_Title", "Title") \
                .withColumnRenamed("_ViewCount", "ViewCount")

postsDF = postsDF \
            .select(["Id", "ParentId", "PostTypeId", "OwnerUserId", \
                     "CreationDate", "Score", "ViewCount", "AnswerCount", \
                     "CommentCount", "LastActivityDate","Tags", "Title", "Body"])

postsDF.printSchema()   
postsDF.show(n=10, truncate=False)

postsDF \
      .write \
      .partitionBy("PostTypeId") \
      .mode("overwrite") \
      .parquet("/content/output/posts.parquet")      

root
 |-- Id: long (nullable = true)
 |-- ParentId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- OwnerUserId: long (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- ViewCount: long (nullable = true)
 |-- AnswerCount: long (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- LastActivityDate: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)

+---+--------+----------+-----------+-----------------------+-----+---------+-----------+------------+-----------------------+---------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### LOAD AND SAVE BADGES.XML DATASET TO PARQUET FORMAT

In [2]:
schema = StructType([ \
    StructField("_Class", LongType(), True), \
    StructField("_Date", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_Name", StringType(), True), \
    StructField("_TagBased", BooleanType(), True), \
    StructField("_UserId", LongType(), True)    
])

badgesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='badges') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Badges.xml')

badgesDF = badgesDF \
                .withColumnRenamed("_Class", "Class") \
                .withColumnRenamed("_Date", "Date") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_Name", "Name") \
                .withColumnRenamed("_TagBased", "TagBased") \
                .withColumnRenamed("_UserId", "UserId")

badgesDF = badgesDF \
              .select(["Id", "UserId", "Class", "Date", "CommentCount", \
                      "TagBased","Name"])

badgesDF.printSchema()
badgesDF.show(n=10, truncate=False)

badgesDF \
      .write \
      .partitionBy("Class") \
      .mode("overwrite") \
      .parquet("/content/output/badges.parquet")

root
 |-- Id: long (nullable = true)
 |-- UserId: long (nullable = true)
 |-- Class: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- TagBased: boolean (nullable = true)
 |-- Name: string (nullable = true)

+---+------+-----+-----------------------+------------+--------+--------------+
|Id |UserId|Class|Date                   |CommentCount|TagBased|Name          |
+---+------+-----+-----------------------+------------+--------+--------------+
|1  |1     |3    |2012-01-17T20:56:42.617|null        |false   |Autobiographer|
|2  |2     |3    |2012-01-17T20:56:42.617|null        |false   |Autobiographer|
|3  |3     |3    |2012-01-17T20:56:42.617|null        |false   |Autobiographer|
|4  |8     |3    |2012-01-17T21:01:42.390|null        |false   |Autobiographer|
|5  |7     |3    |2012-01-17T21:06:42.367|null        |false   |Analytical    |
|6  |11    |3    |2012-01-17T21:06:42.417|null        |false   |Autobiographer|
|7  |20    |3   

#### LOAD AND SAVE COMMENTS.XML DATASET TO PARQUET FORMAT

In [3]:
schema = StructType([ \
    StructField("_ContentLicense", StringType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Text", StringType(), True), \
    StructField("_UserDisplayName", StringType(), True), \
    StructField("_UserId", LongType(), True)        
])

commentsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='comments') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Comments.xml')

commentsDF = commentsDF \
                .withColumnRenamed("_ContentLicense", "ContentLicense") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_PostId", "PostId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Text", "Text") \
                .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
                .withColumnRenamed("_UserId", "UserId") \

commentsDF = commentsDF \
  .select(["Id", "PostId", "UserId", "CreationDate", "ContentLicense", \
           "UserDisplayName", "Score", "Text"])

commentsDF.printSchema()
commentsDF.show(n=10, truncate=False)

commentsDF \
    .write \
    .partitionBy("ContentLicense") \
    .mode("overwrite") \
    .parquet("/content/output/comments.parquet")

root
 |-- Id: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- UserId: long (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- Score: long (nullable = true)
 |-- Text: string (nullable = true)

+---+------+------+-----------------------+--------------+---------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |PostId|UserId|CreationDate           |ContentLicense|UserDisplayName|Score|Text                                                                                                                                                                                                                         

#### LOAD AND SAVE POSTHISTORY.XML DATASET TO PARQUET FORMAT

In [11]:
schema = StructType([ \
    StructField("_Comment", StringType(), True), \
    StructField("_ContentLicense", StringType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostHistoryTypeId", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_RevisionGUID", StringType(), True), \
    StructField("_Text", StringType(), True), \
    StructField("_UserDisplayName", StringType(), True), \
    StructField("_UserId", LongType(), True)    
])

postHistoryDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posthistory') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/PostHistory.xml')

               
postHistoryDF = postHistoryDF \
              .withColumnRenamed("_Comment", "Comment") \
              .withColumnRenamed("_ContentLicense", "ContentLicense") \
              .withColumnRenamed("_CreationDate", "CreationDate") \
              .withColumnRenamed("_Id", "Id") \
              .withColumnRenamed("_PostHistoryTypeId", "PostHistoryTypeId") \
              .withColumnRenamed("_PostId", "PostId") \
              .withColumnRenamed("_RevisionGUID", "RevisionGUID") \
              .withColumnRenamed("_Text", "Text") \
              .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
              .withColumnRenamed("_UserId", "UserId") \

postHistoryDF = postHistoryDF \
              .select(["Id", "PostId", "PostHistoryTypeId", "UserId", \
                       "CreationDate", "RevisionGUID", "ContentLicense", "UserDisplayName", "Comment", "Text"])

postHistoryDF.printSchema()
postHistoryDF.show(n=10, truncate=False)  

postHistoryDF \
  .write \
  .partitionBy("ContentLicense") \
  .mode("overwrite") \
  .parquet("/content/output/posthistory.parquet")  

root
 |-- Id: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- PostHistoryTypeId: long (nullable = true)
 |-- UserId: long (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- RevisionGUID: string (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Text: string (nullable = true)

+---+------+-----------------+------+-----------------------+------------------------------------+--------------+---------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### LOAD AND SAVE TAGS.XML DATASET TO PARQUET FORMAT

In [ ]:
tagsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='tags') \
          .options(rowTag='row') \
          .load('/content/input/Tags.xml')

tagsDF.show(n=10)

#### LOAD AND SAVE USERS.XML DATASET TO PARQUET FORMAT

In [ ]:
usersDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='Users') \
          .options(rowTag='row') \
          .load('/content/input/Users.xml')

usersDF.show(n=10) 

#### LOAD AND SAVE VOTES.XML DATASET TO PARQUET FORMAT

In [ ]:
votesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='votes') \
          .options(rowTag='row') \
          .load('/content/input/Votes.xml')

votesDF.show(n=10) 

In [ ]:
### ANALITYKA POSTS

#df.withColumn('word', f.explode(f.split(f.col('Description'), ' ')))\
    #.groupBy('word')\
    #.count()\
    #.sort('count', ascending=False)\
    #.show()